<a href="https://colab.research.google.com/github/olinyoder2534/PaperGrader/blob/main/PaperGrader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn
import numpy as np

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
X_train.shape

(60000, 28, 28)

In [ ]:
X_train_reshaped = np.expand_dims(X_train, axis=-1)

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal",
                                                 input_shape=(28,
                                                              28,
                                                              1)),
    layers.experimental.preprocessing.RandomRotation(0.25),
    layers.experimental.preprocessing.RandomZoom(0.25),
  ]
)


In [ ]:
model1 = models.Sequential([
    layers.Conv2D(filters=50, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=100, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    #layers.Conv2D(filters=150, kernel_size=(3, 3), activation='relu'),
    #layers.MaxPooling2D((2, 2)),

    layers.Dropout(.25),

    layers.Flatten(),
    layers.Dense(100, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model1.fit(X_train_reshaped/255, y_train, epochs=6)

Epoch 1/6
1875/1875 [==============================] - 96s 51ms/step - loss: 0.1266 - accuracy: 0.9613
Epoch 2/6
1875/1875 [==============================] - 91s 49ms/step - loss: 0.0454 - accuracy: 0.9856
Epoch 3/6
1875/1875 [==============================] - 91s 49ms/step - loss: 0.0325 - accuracy: 0.9896
Epoch 4/6
1875/1875 [==============================] - 90s 48ms/step - loss: 0.0247 - accuracy: 0.9924
Epoch 5/6
1875/1875 [==============================] - 91s 48ms/step - loss: 0.0208 - accuracy: 0.9935
Epoch 6/6
1875/1875 [==============================] - 96s 51ms/step - loss: 0.0166 - accuracy: 0.9948


In [ ]:
model1.evaluate(X_test/255, y_test)

313/313 [==============================] - 6s 18ms/step - loss: 0.0253 - accuracy: 0.9923


[0.025336816906929016, 0.9922999739646912]

In [ ]:
model1.save('model1.h5')

In [ ]:
from keras.models import load_model
model1 = load_model('model1.h5')

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from google.colab.patches import cv2_imshow

model1 = load_model('model1.h5')

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (28, 28), interpolation=cv2.INTER_AREA)
    processed_image = np.expand_dims(resized, axis=-1)
    processed_image = processed_image / 255
    return processed_image

def identify_numbers(image):
    processed_image = preprocess_image(image)
    predictions = model1.predict(processed_image.reshape(-1, 28, 28, 1))
    predicted_numbers = [np.argmax(prediction) for prediction in predictions]
    return predicted_numbers

def check_answers(correct_answers, identified_numbers):
    num_correct = 0
    num_questions = len(correct_answers)
    correct_questions = []
    wrong_questions = []
    for i, (correct_answer, identified_number) in enumerate(zip(correct_answers, identified_numbers), start=1):
        if correct_answer == identified_number:
            num_correct += 1
            correct_questions.append(i)
        else:
            wrong_questions.append(i)
    return num_correct, num_questions, correct_questions, wrong_questions

def main(num_questions, correct_answers):
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        if frame is None:
            print("Error: Unable to read frame from the camera.")
            break

        cv2_imshow(frame)

        identified_numbers = identify_numbers(frame)

        num_correct, num_questions, correct_questions, wrong_questions = check_answers(correct_answers, identified_numbers)

        print(f"Number of correct answers: {num_correct}/{num_questions}")
        print(f"Correct questions: {correct_questions}")
        print(f"Wrong questions: {wrong_questions}")

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    num_questions = 20
    correct_answers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0]

    main(num_questions, correct_answers)

Error: Unable to read frame from the camera.
